<b>programmer_guide</b> is a more detailed documentation, that also explores the basic building blocks of the existing simulation tools- if you wish to build your own functions, improve on the program, or test significant changes to the system.

* You should read <b>user_guide</b> first, and play around with <b>light_guide</b> before getting into this one. Also, I recommend to check out [sympy.Vector](https://docs.sympy.org/latest/modules/vector/index.html).

* When explaining functions, I'll try to keep it simple and focus on input+output. When possible, I'll try to refrain from explaining the math/logic behind the function. 

* If you are familiar with the law of reflection (very basic optics), comfortable with vectors, and know some multivariable calculus- you should mostly be able to figure out the functions on your own. The comments '#' throughout will help you.

* If there's a problem with the original code, if you've built a new feature that would be useful to other people, or you're really stuck- shoot me an email at oren.ir49@gmail.com or orenir@post.bgu.ac.il.

* <i>Most importantly- never change anything in the code, that you don't know how to change back.</i>


# First Cell
* Only imports here, pretty straight forward. The sympy lines may be odd to you at first, so:
    * 'from sympy import *'- that means you can use sympy objects and functions, without explicitly refering to the sympy library. For example, instead of writing 'sympy.pi' you can just write 'pi' and the interpreter will know what you mean.
    * 'interactive.printing.init_printing(use_latex=True)'- that gives you the ability to print out sympy objects with latex fonts. For example, try to call 'print(pi)', pprint(pi), and the latex option 'display(pi)'.
    * 'N = ReferenceFrame('N')'- this was mentioned in <b>user_guide</b>, and on 'sympy.Vector', but I'll explain briefly- this defines 'N' as our coordinate system, s.t we can access 3 unit vectors: ' N.x, N.y, N.z ' and also their respective coordinate variables ' N[0], N[1], N[2] '. Every surface you create should be an expression involving the ' N[i] ' variables, and every vector you use should be a linear combination of the ' N.x '-s.

# Second Cell
* This is the 'Detector Construction' cell. It's covered in detail in <b>user_guide</b> and also with comments throughout.

# Third Cell

## Basic ray tracing
* <b>Ray_Generator_ZDC()</b> simulates a single photon, based on no input. It uses the user defined grids of 'source_grid' (for x,y positions) and 'offset_grid' (for z positions). Together with the fiber radius 'rf', these pre-defined parameters control the spatial distribution for the photons- the photon always comes out of a point, on the circular surface of one of the rods. The angular distribution is defined inside this function. 'sanity_check_sources(rays)' sums it ups pretty well.
    * Returns 'ps'- a point for the source, 'es'- a unit vector for the trajectory, and '(nx,ny)'- identification of the emitting rod.
* <b>Ray_Generator_Parallel()</b> simulates a single photon, traveling in parallel to the ' N.z ' axis. Mainly useful for sanity checks. The spatial distribution is uniform on a rectangle (-xb,xb)x(-yb,yb) on the z=0 plane.
    * Re
* <b>ray_x,ray_y</b> mainly help plotting, they take in 'p' and 'e'- point and unit vector that characterize a light ray, and return x and y (respectively) at the point z=z. This of course wouldn't help for horizontal light rays (parallel to xy plane).
* <b>Intersection(surface,p,e,bottom,top)</b> takes in a surface of the form  $f\begin{pmatrix}x \\ y \\z \end{pmatrix}=0$, and solves analytically for the intersection point of a light ray 'p,e' with 'surface' in the range 'bottom<z<top'. 
    * This function returns a point of intersection 'xi*N.x + yi*N.y + zi*N.z' if one has been found, or '0*N.x' otherwise. 
    * This function was made obsolete, by <b>Bisection(surface,p,e,h)</b>. It's very slow, and also can't solve complicated equations.
* <b>Bisection(surface,p,e,height)</b> has almost the exact same input and output of <b>Intersection(surface,p,e,bottom,top)</b>. Here 'bottom = 0' by default, and 'height' plays the part of 'top'. It uses the '[false position](https://en.wikipedia.org/wiki/Regula_falsi)' method- which I'd chosen for reliability, flexibility, speed and accuracy.
    * You can tinker with the tolerance 'atol' and the maximum 'index=20', and also with the initial bracket 'gl=0,gr=2*height'. These allow you trade accuracy and security with speed- I chose parameters that guarantee very high accuracy, and offer great reliability (e.g if a root exists, it wiil be found), at the cost of some speed. However, I was still very satisfied with the run time.
* <b>Dog_Cone(ps,es,tup)</b> takes in a light ray 'ps,es' and a tuplet '(nx,ny)' (ID for a quartz fiber on the grid). Returns new 'ps,es' for the light ray- which may or may have not been deflected by a dog cone attached to the fiber.
    * Assumes a dog cone exists on the rod. If working with 'cone_grid' you should check before calling this function.
* <b>Normal_Vec(surface,intersect)</b> takes in a parametrized surface $z= f\begin{pmatrix}x \\ y\end{pmatrix}$ and a point on the surface- and return the unit normal vector to the surface, at that point. 
    * This function has was made obsolete, by  obsolete, by <b>Grad_Vec(surface,intersect)</b> which is much, much faster.
* <b>Grad_Vec(surface,intersect)</b> takes in a surface $f\begin{pmatrix}x \\ y \\z \end{pmatrix}=0$ and a point on the surface- and returns the unit normal vector to the surface, at that point.
    * The surface is given to us as a level set of a multivariable function. It's known that the gradient of the function, is perpendicular to level sets- and that allows us to find the normal vector immidiately.
* <b>Ref_Vec(e,n)</b> takes in the unit vector 'e' (trajectory for incident ray), and the unit vector 'n' (normal vector at the point of intersection) and returns a unit vector 'r' which is the trajectory for the reflected ray.
    * The vector 'n' is the bisector of 'e' and 'r'. The diagonal in a rhombus, is the bisector for the sides. I derived the expression for 'r' by building a rhombus from 'e' and 'r', and requiring '$\lambda \cdot \overrightarrow{n}$' to be the diagonal, where '$\lambda$' is a scalar and '$\overrightarrow{n}$' is the known, normalized, normal vector. 
    * The diagonal requirement, and the requirement for 'e,r,n' to be normalized- is sufficient to find $\lambda$.
* <b>Is_Absorbed(p,e)</b> takes in a light ray 'p,e', returns True is this ray passes through the PMT defined by (-xp,xp)x(-yp,yp)- or False otherwise.

## Simulation functions

* <b>sanity_check_..(...)</b> those 3 functions are covered in <b>user_guide</b>
* <b>Test_Efficiency_Background(eq,num,q,N_bounce,parallel,cones)</b> this function is intended to be called by <b>Test_Efficiency</b>. This handles the ray tracing process- it takes the surface level set 'eq', and simulates 'num' rays that hit it and are allowed to bounce up to 'N_bounce' times (or, until the ray trajectory flips- and it starts traveling downard). 'q' is a feature of the python ['multiprocessing'](https://docs.python.org/3/library/multiprocessing.html) library, in this instance we pass 'q' onto the background function, which enters the number of detected rays- to be used by <b>Test_Efficiency</b>.
    * Instead of explaining the order of operations in this function, I suggest you read it line by line- and draw a flow diagram. It should be fairly intuitive
* <b>Test_Efficiency(eq,N_iter=100,N_ray=100,N_proc=4,N_bounce=1,parallel=False,cones=True)</b> is explained in detail in <b>user_guide</b>. I want to note the order of operations when using multiprocessing- first you create an empty list of processes, then you start the processes one by one and save them on a list. Then you first extract information off of the Queues you passed on- and only then, call process.join() for all of the processes you created. 
    * If you wish to write a function that uses multiprocessing similarily, I suggest using this one as a template; and the explaination above as a guideline. It's very simple and easy to read.
    * This function not only draws a histogram (with mean and standard deviation of the mean printed on it), but also returns an 'N_iter' length array, containing the PMT counter from each batch of rays simulated.
* <b>Required_Gradient(e,intersect)</b> takes the incident ray trajectory 'e', and the intersection point with some boundary. It does reverse engineering to <b>Ref_Vec()</b>- by requiring that the reflected photon hits the PMT, the function finds a suitable normal vector at the intersection point (up to multiplying by a scalar).
    * The function returns a quantity which doesn't depend on the magnitude of the normal vector: the ratio between the (cylindrical) '$\rho$' component and the 'z' component. (1e-4 in the denominator- to avoid division by 0 in rare cases).
    * This quantity is important to function fitting, by the implicit function theorem- it is the implicit derivative '$\frac{dz}{d\rho}$' at the point of intersection. Comments throughout the code explain, and also <b>user_guide</b>.
    * It could be interesting to extract also cylindrical '$\phi$' components of the gradient. That could tell us something about the validity of the radial symmetry assumption in optimization. I assume, that after averaging over a very large number of rays- different'$\phi$' measurments will tend to cancel, leaving the averaged component negligble to 'z','$\rho$'.

* <b>Binning(data,bins)</b> takes a 3 column matrix 'data', in which the first column are measurments of 'z' position (intersection points of photons with surface), the second column has measurments of 'rho' position, and the third has the matching, implicit '$\frac{dz}{d\rho}$' given by the function above. The function sorts 'data' with respect to 'z' column, divides the 'z' range into the given number of intervals ('bins'), and then creates a new (binsx6) matrix of averaged measurments (and standard deviations). The columns will be ($z$,$\delta z$,$\rho$,$\delta\rho$,$\frac{dz}{d\rho}$,$\delta\frac{dz}{d\rho}$).
    * The measurments tend to bunch up, sometimes leaving intervals empty or with a single measurment. Expect standard deviations of 0 on occasion.
* <b>Optimize_Background(eq,N_ray,parallel,cones,qe,q1,q2,q3)</b> simulates 'N_ray' photons on surface 'eq' and uses the Queues 'qe,q1,q2,q3' to pass on measurments of efficiency,'z','$\rho$',and '$\frac{dz}{d\rho}$' respectively. The logic is very similar to <b>Test_Efficiency_Background()</b>, if you read that function like I suggested- this one should make sense. This function is intended to be used by <b>Optimize(...)</b>, and its job is to produce data for function fitting.
* <b>Optimize(...)</b> is covered pretty well by <b>user_guide</b> and also has detailed comments throughout. In part, it's very similar to <b>Test_Efficiency(...)</b> in terms of multiprocessing work. The other part of this function, has to do with the fitting. The data from <b>Optimize_Background(...)</b> is cleaned up by <b>Binning(...)</b> and sent to scipy.optimize for regression to a function $\frac{dz}{d\rho}$. If the user made sure everything is consistent, the fitting will give us all the needed parameters to describe the surface $f\begin{pmatrix}\rho \\z \end{pmatrix}=0$ itself (except for a constant of integration which is worked out based on a constraint we can control).
    * The function returns 'ef_list' which stores efficiency values for each surface from the process, 'surface_list' which stores the surface equations themself, and 'coef_mat' which stores the coefficients that describe the surfaces- useful for convergence plots.
    * <u>In some places in this function, the comments have 3 '###'. This indicates places where the user needs to check for consistency, if changes are made to the functional form of the surface we are trying to fit.</u>
* <b>Optimization_Results(ef_list,surface_list,coef_mat)</b> is intended to present the output from <b>Optimize(...)</b> nicely. More detail in <b>user_guide</b>, the function itself is very straightforward and the user can always change the style of the plots.